# FEniCS simulation of Eshelby's circular inclusion problem

The aim of this notebook is to setup a very simple FEniCS simulation. The framework is linear, plane strain elasticity. We model a matrix in a disk around the origin (radius $R_m$) with an inclusion having the shape of another disk around the origin, with a smaller radius ($R_i < R_e$). The matrix and the inclusion have different elastic modulus ($E$: Young modulus; $\nu$: Poisson ratio) but are both isotropic and linearly elastic.

\begin{equation}
\sigma_{ij} = \lambda\varepsilon_{kk}\delta_{ij}+2\mu\varepsilon_{ij},
\end{equation}

where indices $i, j, k$ are restricted to $\{1, 2\}$ and $\lambda$, $\mu$ are the Lamé coefficients :

\begin{equation*}
\mu=\frac{E}{2\bigl(1+\nu\bigr)}
\quad\text{and}\quad
\lambda=\frac{2\mu\nu}{1-2\nu}.
\end{equation*}

The variational formulation of the problem is the following:

Find $u\in \mathcal{C}\equiv\{u: H^1(\Omega), \; u(x_1,x_2)|_{x_1^2+x_2^2=R_e^2}
%\text{border}
=(-x_2,-x_1)\}$ such that 
$\forall v\in \mathcal{C}_0\equiv \mathcal{C}$


\begin{equation}
\int_\Omega \sigma(\varepsilon(u)):\varepsilon(v)\,\mathrm{d}x\,\mathrm{d}y =
\int_{\Omega} b \cdot v\,\mathrm{d} x\,\mathrm{d} y,
\end{equation}

where the body force $b=0$ and $\sigma(\varepsilon)$ is the constitutive equation and $\varepsilon(u)=\mathrm{sym} (\nabla u)$  

![shema](inclusion_shear.png)

you can find help here:

- Mesh generation
   - https://jorgensd.github.io/dolfinx-tutorial/chapter1/membrane_code.html
   - https://jorgensd.github.io/dolfinx-tutorial/chapter2/ns_code2.html#mesh-generation
   - https://docs.fenicsproject.org/dolfinx/main/python/demos/demo_gmsh.html
- Dirichlet bc
    - https://jorgensd.github.io/dolfinx-tutorial/chapter1/fundamentals_code.html#defining-the-boundary-conditions
    - https://jorgensd.github.io/dolfinx-tutorial/chapter2/linearelasticity_code.html#boundary-conditions
    - https://jorgensd.github.io/dolfinx-tutorial/chapter3/component_bc.html
- Visualization in Paraview
    - https://jorgensd.github.io/dolfinx-tutorial/chapter1/membrane_paraview.html
- Interpolating the strain tensor once we have the displacement vector
    - https://jorgensd.github.io/dolfinx-tutorial/chapter2/linearelasticity_code.html#stress-computation
- pyvista
    - https://jorgensd.github.io/dolfinx-tutorial/chapter2/linearelasticity_code.html#visualization
    - https://docs.fenicsproject.org/dolfinx/main/python/demos/demo_pyvista.html
- The Assemble function to perform integrals
    - https://jorgensd.github.io/dolfinx-tutorial/chapter1/fundamentals_code.html#computing-the-error
- Convergence study
    - https://jorgensd.github.io/dolfinx-tutorial/chapter4/convergence.html

In [1]:
import dolfinx # FEM in python
import matplotlib.pyplot as plt
import ufl # variational formulations
import numpy as np
from mpi4py import MPI
from petsc4py.PETSc import ScalarType
import gmsh # Mesh generation
import dolfinx.mesh as mesh #from fracture 

In [2]:
import pyvista # visualisation in python notebook
# ONLY EXCUTE ONCE if needed for pyvista
# !pip install ipyvtklink==0.2.3 

In [3]:
import extract # this to be able to have the value of a solution at point (x,y)
# use it this way : extract.solution(my_domain, u_solution, x=0.5, y=0.01)

# Parameters of the simulation

In [4]:
def solve_inclusion(
    R_i = 1.0, # Radius of the inclusion
    R_e = 6.9,  # Radius of the matrix (whole domain)
    aspect_ratio = 1.0, # start with a circle, otherwise ellipse)
    E_m = 1.0, # Young's modulus in matrix
    nu_m = 0.35, # Poisson's ratio in matrix
    E_i = 11.0, # Young's modulus of inclusion
    nu_i = 0.3, # Poisson's ratio in inclusion
    mesh_size = 1/3,
    mesh_order = 1 
    #solution_number=1
):
    mesh_comm = MPI.COMM_WORLD
    model_rank = 0
    gmsh.initialize()
    facet_names = {"inner_boundary": 1, "outer_boundary": 2}
    cell_names = {"inclusion": 1, "matrix": 2}
    model = gmsh.model()
    model.add("Disk")
    model.setCurrent("Disk")
    gdim = 2 # geometric dimension of the mesh
    inner_disk = gmsh.model.occ.addDisk(0, 0, 0, R_i, aspect_ratio * R_i)
    outer_disk = gmsh.model.occ.addDisk(0, 0, 0, R_e, R_e)
    whole_domain = gmsh.model.occ.fragment([(gdim, outer_disk)], [(gdim, inner_disk)])
    gmsh.model.occ.synchronize()
    
    # Add physical tag for bulk
    inner_domain = whole_domain[0][0]
    outer_domain = whole_domain[0][1]
    model.addPhysicalGroup(gdim, [inner_domain[1]], tag=cell_names["inclusion"])
    model.setPhysicalName(gdim, inner_domain[1], "Inclusion")
    model.addPhysicalGroup(gdim, [outer_domain[1]], tag=cell_names["matrix"])
    model.setPhysicalName(gdim, outer_domain[1], "Matrix")
    
    # Add physical tag for boundaries
    lines = gmsh.model.getEntities(dim=1)
    inner_boundary = lines[1][1]
    outer_boundary = lines[0][1]
    gmsh.model.addPhysicalGroup(1, [inner_boundary], facet_names["inner_boundary"])
    gmsh.model.addPhysicalGroup(1, [outer_boundary], facet_names["outer_boundary"])
    gmsh.option.setNumber("Mesh.CharacteristicLengthMin",mesh_size)
    gmsh.option.setNumber("Mesh.CharacteristicLengthMax",mesh_size)
    model.mesh.generate(gdim)
    gmsh.option.setNumber("General.Terminal", 1)
    model.mesh.setOrder(mesh_order)
    gmsh.option.setNumber("General.Terminal", 0)
    
    # Import the mesh in dolfinx
    from dolfinx.io import gmshio
    domain, cell_tags, facet_tags = gmshio.model_to_mesh(model, mesh_comm, model_rank, gdim=gdim)
    domain.name = "composite"
    cell_tags.name = f"{domain.name}_cells"
    facet_tags.name = f"{domain.name}_facets"
    gmsh.finalize()
    
    from dolfinx.plot import create_vtk_mesh
    
    
    ds = ufl.Measure("ds", domain=domain, subdomain_data=facet_tags) # on précise aussi le numéro de sous-domaine avec facet_tags
    dx = ufl.Measure("dx", domain=domain, subdomain_data=cell_tags)   # idem
    one = dolfinx.fem.Constant(domain,ScalarType(1.))
    
    area_inclusion = dolfinx.fem.assemble_scalar(dolfinx.fem.form(one * dx(1)))
    area_matrix = dolfinx.fem.assemble_scalar(dolfinx.fem.form(one * dx(2)))
    area_domain = dolfinx.fem.assemble_scalar(dolfinx.fem.form(one * ufl.dx))
    
    V = dolfinx.fem.VectorFunctionSpace(domain,("Lagrange", 1),dim=2)

    def eps(u):
        return ufl.sym(ufl.grad(u))
    
    I2 = ufl.Identity(2)
    
    # Hooke's law is written as the top of this notebook
    def sigma(eps, E, nu):
        mu = E/(2*(1+nu))
        lamb = 2*mu*nu/(1-2*nu)
        return lamb*ufl.tr(eps)*I2 + 2*mu*eps
    
    u = ufl.TrialFunction(V)
    u_bar = ufl.TestFunction(V)
    
    bilinear_form_inclusion = ufl.inner(sigma(eps(u), E_i, nu_i), eps(u_bar))*dx(1)
    bilinear_form_matrix = ufl.inner(sigma(eps(u), E_m, nu_m), eps(u_bar))*dx(2)
    bilinear_form = bilinear_form_inclusion + bilinear_form_matrix
    
    g=0.0 # no weight
    body_force = dolfinx.fem.Constant(domain, ScalarType((0,-g)))
    linear_form = ( ufl.dot(body_force,u_bar)  ) * ufl.dx
    
    outer_facets = facet_tags.find(2)
    outer_boundary_dofs = dolfinx.fem.locate_dofs_topological(V, domain.topology.dim - 1, outer_facets)

    uD = dolfinx.fem.Function(V)
    u_on_boundary = lambda x: np.array([-x[1], -x[0]], dtype=ScalarType)
    uD.interpolate(u_on_boundary)
    bc = dolfinx.fem.dirichletbc(uD, outer_boundary_dofs)
    
    problem = dolfinx.fem.petsc.LinearProblem(bilinear_form, linear_form, bcs=[bc], 
                                          petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
    u_solution = problem.solve()
    
    from eshelby import EshelbyDisk
    solution = EshelbyDisk(V,R_e/R_i, E_i/E_m, nu_i, nu_m)
    u_ref_func = solution.to_function(R_i)
    
    error = np.sqrt( dolfinx.fem.assemble_scalar(dolfinx.fem.form( ufl.dot(u_solution-u_ref_func,u_solution-u_ref_func)* ufl.dx)) )

    
    return u_solution, u_ref_func, error
    

In [5]:
u_solution, u_ref_func, error = solve_inclusion(R_i = 1.0, # Radius of the inclusion
    R_e = 6.9,  # Radius of the matrix (whole domain)
    aspect_ratio = 1.0, # start with a circle, otherwise ellipse)
    E_m = 1.0, # Young's modulus in matrix
    nu_m = 0.35, # Poisson's ratio in matrix
    E_i = 11.0, # Young's modulus of inclusion
    nu_i = 0.3, # Poisson's ratio in inclusion
    mesh_size = 1/3,
    mesh_order = 1 )



Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Ellipse)
Info    : [ 50%] Meshing curve 2 (Ellipse)
Info    : Done meshing 1D (Wall 0.00141951s, CPU 0.002296s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 50%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0991388s, CPU 0.099744s)
Info    : 1689 nodes 3397 elements


In [6]:
# Material
L2_sol = np.sqrt( dolfinx.fem.assemble_scalar(dolfinx.fem.form( ufl.dot(u_solution,u_solution)* ufl.dx)) )
L2_ref = np.sqrt( dolfinx.fem.assemble_scalar(dolfinx.fem.form( ufl.dot(u_ref_func,u_ref_func)* ufl.dx)) )
print(L2_sol)
print(L2_ref)
print(error)

59.01888789899557
59.03159668097725
0.052965677904016975


# 12) Convergence study

In [7]:
#in that part E_i/E_m = 11; nu_m = 0.35; nu_i = 0.3

In [8]:
# degree = 1
mesh_values = [1, 0.5, 0.25, 0.125, 0.0625]
i_values = [1.0, 2.0, 3.0, 4.0, 5.0]
error_vect=np.array(i_values)
for i in range(5):
    
    u_solution, u_ref_func, error = solve_inclusion(R_i = 1.0, # Radius of the inclusion
        R_e = 6.9,  # Radius of the matrix (whole domain)
        aspect_ratio = 1.0, # start with a circle, otherwise ellipse)
        E_m = 1.0, # Young's modulus in matrix
        nu_m = 0.35, # Poisson's ratio in matrix
        E_i = 11.0, # Young's modulus of inclusion
        nu_i = 0.3, # Poisson's ratio in inclusion
        mesh_size = mesh_values[i],
        mesh_order = 1 )

    L2_sol = np.sqrt( dolfinx.fem.assemble_scalar(dolfinx.fem.form( ufl.dot(u_solution,u_solution)* ufl.dx)) )
    L2_ref = np.sqrt( dolfinx.fem.assemble_scalar(dolfinx.fem.form( ufl.dot(u_ref_func,u_ref_func)* ufl.dx)) )
    print(L2_sol)
    print(L2_ref)
    error_vect[i]=error
    
print(error_vect)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Ellipse)
Info    : [ 50%] Meshing curve 2 (Ellipse)
Info    : Done meshing 1D (Wall 0.00071488s, CPU 0.000985s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 50%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0121138s, CPU 0.012176s)
Info    : 232 nodes 471 elements
58.81705561458823
58.83891107138668
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Ellipse)
Info    : [ 50%] Meshing curve 2 (Ellipse)
Info    : Done meshing 1D (Wall 0.000422795s, CPU 0.001098s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 50%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0307512s, CPU 0.050017s)
Info    : 801 nodes 1615 elements
58.983314276364204
59.000074721848975
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Ellipse)
Info    : [ 50%] Meshing curve 2 (Ellipse

In [9]:
# degree = 2
mesh_values = [1, 0.5, 0.25, 0.125, 0.0625]
i_values = [1.0, 2.0, 3.0, 4.0, 5.0]
error_vect=np.array(i_values)
for i in range(5):
    
    u_solution, u_ref_func, error = solve_inclusion(R_i = 1.0, # Radius of the inclusion
        R_e = 6.9,  # Radius of the matrix (whole domain)
        aspect_ratio = 1.0, # start with a circle, otherwise ellipse)
        E_m = 1.0, # Young's modulus in matrix
        nu_m = 0.35, # Poisson's ratio in matrix
        E_i = 11.0, # Young's modulus of inclusion
        nu_i = 0.3, # Poisson's ratio in inclusion
        mesh_size = mesh_values[i],
        mesh_order = 2 )

    L2_sol = np.sqrt( dolfinx.fem.assemble_scalar(dolfinx.fem.form( ufl.dot(u_solution,u_solution)* ufl.dx)) )
    L2_ref = np.sqrt( dolfinx.fem.assemble_scalar(dolfinx.fem.form( ufl.dot(u_ref_func,u_ref_func)* ufl.dx)) )
    print(L2_sol)
    print(L2_ref)
    error_vect[i]=error
    
print(error_vect)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Ellipse)
Info    : [ 50%] Meshing curve 2 (Ellipse)
Info    : Done meshing 1D (Wall 0.000530213s, CPU 0.000804s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 50%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0132244s, CPU 0.013317s)
Info    : 232 nodes 471 elements
Info    : Meshing order 2 (curvilinear on)...
Info    : [  0%] Meshing curve 1 order 2
Info    : [ 30%] Meshing curve 2 order 2
Info    : [ 50%] Meshing surface 1 order 2
Info    : [ 80%] Meshing surface 2 order 2
Info    : Surface mesh: worst distortion = 0.682077 (0 elements in ]0, 0.2]); worst gamma = 0.799327
Info    : Done meshing order 2 (Wall 0.00189459s, CPU 0.002112s)
58.85629675530022
59.03111265931016
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Ellipse)
Info    : [ 50%] Meshing curve 2 (Ellipse)
Info    : Done meshing 1D (Wall 0.000414401s, CPU 0.000489s)
Info

In [10]:
#Convergence plot 

In [11]:
#E_i/E_m = 110

In [12]:
mesh_values = [1, 0.5, 0.25, 0.125, 0.0625]
i_values = [1.0, 2.0, 3.0, 4.0, 5.0]
error_vect=np.array(i_values)
for i in range(5):
    
    u_solution, u_ref_func, error = solve_inclusion(R_i = 1.0, # Radius of the inclusion
        R_e = 6.9,  # Radius of the matrix (whole domain)
        aspect_ratio = 1.0, # start with a circle, otherwise ellipse)
        E_m = 1.0, # Young's modulus in matrix
        nu_m = 0.35, # Poisson's ratio in matrix
        E_i = 110.0, # Young's modulus of inclusion
        nu_i = 0.3, # Poisson's ratio in inclusion
        mesh_size = mesh_values[i],
        mesh_order = 1 )

    L2_sol = np.sqrt( dolfinx.fem.assemble_scalar(dolfinx.fem.form( ufl.dot(u_solution,u_solution)* ufl.dx)) )
    L2_ref = np.sqrt( dolfinx.fem.assemble_scalar(dolfinx.fem.form( ufl.dot(u_ref_func,u_ref_func)* ufl.dx)) )
    print(L2_sol)
    print(L2_ref)
    error_vect[i]=error
    
print(error_vect)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Ellipse)
Info    : [ 50%] Meshing curve 2 (Ellipse)
Info    : Done meshing 1D (Wall 0.00128819s, CPU 0.002194s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 50%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0272549s, CPU 0.027289s)
Info    : 232 nodes 471 elements
58.70981921308258
58.75659373068899
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Ellipse)
Info    : [ 50%] Meshing curve 2 (Ellipse)
Info    : Done meshing 1D (Wall 0.000430014s, CPU 0.000518s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 50%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0365633s, CPU 0.036588s)
Info    : 801 nodes 1615 elements
58.89293439743808
58.91970448828552
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Ellipse)
Info    : [ 50%] Meshing curve 2 (Ellipse)


In [13]:
#E_i/E_m = 1100

In [14]:
mesh_values = [1, 0.5, 0.25, 0.125, 0.0625]
i_values = [1.0, 2.0, 3.0, 4.0, 5.0]
error_vect=np.array(i_values)
for i in range(5):
    
    u_solution, u_ref_func, error = solve_inclusion(R_i = 1.0, # Radius of the inclusion
        R_e = 6.9,  # Radius of the matrix (whole domain)
        aspect_ratio = 1.0, # start with a circle, otherwise ellipse)
        E_m = 1.0, # Young's modulus in matrix
        nu_m = 0.35, # Poisson's ratio in matrix
        E_i = 1100.0, # Young's modulus of inclusion
        nu_i = 0.3, # Poisson's ratio in inclusion
        mesh_size = mesh_values[i],
        mesh_order = 1 )

    L2_sol = np.sqrt( dolfinx.fem.assemble_scalar(dolfinx.fem.form( ufl.dot(u_solution,u_solution)* ufl.dx)) )
    L2_ref = np.sqrt( dolfinx.fem.assemble_scalar(dolfinx.fem.form( ufl.dot(u_ref_func,u_ref_func)* ufl.dx)) )
    print(L2_sol)
    print(L2_ref)
    error_vect[i]=error
    
print(error_vect)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Ellipse)
Info    : [ 50%] Meshing curve 2 (Ellipse)
Info    : Done meshing 1D (Wall 0.00132643s, CPU 0.001967s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 50%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0218009s, CPU 0.022098s)
Info    : 232 nodes 471 elements
58.69821851624347
58.748005574643265
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Ellipse)
Info    : [ 50%] Meshing curve 2 (Ellipse)
Info    : Done meshing 1D (Wall 0.000411726s, CPU 0.000515s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 50%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0341955s, CPU 0.034201s)
Info    : 801 nodes 1615 elements
58.88337623896678
58.91132910973001
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Ellipse)
Info    : [ 50%] Meshing curve 2 (Ellipse)

In [15]:
#E_i/E_m = 0.001 ! cas sans inclusion

In [16]:
mesh_values = [1, 0.5, 0.25, 0.125, 0.0625]
i_values = [1.0, 2.0, 3.0, 4.0, 5.0]
error_vect=np.array(i_values)
for i in range(5):
    
    u_solution, u_ref_func, error = solve_inclusion(R_i = 1.0, # Radius of the inclusion
        R_e = 6.9,  # Radius of the matrix (whole domain)
        aspect_ratio = 1.0, # start with a circle, otherwise ellipse)
        E_m = 1.0, # Young's modulus in matrix
        nu_m = 0.35, # Poisson's ratio in matrix
        E_i = 0.001, # Young's modulus of inclusion
        nu_i = 0.3, # Poisson's ratio in inclusion
        mesh_size = mesh_values[i],
        mesh_order = 1 )

    L2_sol = np.sqrt( dolfinx.fem.assemble_scalar(dolfinx.fem.form( ufl.dot(u_solution,u_solution)* ufl.dx)) )
    L2_ref = np.sqrt( dolfinx.fem.assemble_scalar(dolfinx.fem.form( ufl.dot(u_ref_func,u_ref_func)* ufl.dx)) )
    print(L2_sol)
    print(L2_ref)
    error_vect[i]=error
    
print(error_vect)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Ellipse)
Info    : [ 50%] Meshing curve 2 (Ellipse)
Info    : Done meshing 1D (Wall 0.00101294s, CPU 0.001621s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 50%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.021493s, CPU 0.021389s)
Info    : 232 nodes 471 elements
60.338445035623316
60.75009205150293
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Ellipse)
Info    : [ 50%] Meshing curve 2 (Ellipse)
Info    : Done meshing 1D (Wall 0.000413337s, CPU 0.000444s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 50%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0331677s, CPU 0.032979s)
Info    : 801 nodes 1615 elements
60.711118820439566
60.89405846136707
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Ellipse)
Info    : [ 50%] Meshing curve 2 (Ellipse)

In [17]:
# nu_i = 0.3, 0.35, 0.4, 0.45, 0.49 
error_matrix= []

In [19]:
# degree = 1
mesh_values = [1, 0.5, 0.25, 0.125, 0.0625]
nu_i_values = [0.3, 0.35, 0.4, 0.45, 0.49, 0.499, 0.4999]
i_values = [1.0, 2.0, 3.0, 4.0, 5.0]
error_matrix = [[1, 2, 3, 4, 5, 6, 7],[1, 2, 3, 4, 5, 6, 7],[1, 2, 3, 4, 5, 6, 7],[1, 2, 3, 4, 5, 6, 7],[1, 2, 3, 4, 5, 6, 7],[1, 2, 3, 4, 5, 6, 7],[1, 2, 3, 4, 5, 6, 7]]
for i in range(7):
    for j in range(5):
    
        u_solution, u_ref_func, error = solve_inclusion(R_i = 1.0, # Radius of the inclusion
            R_e = 6.9,  # Radius of the matrix (whole domain)
            aspect_ratio = 1.0, # start with a circle, otherwise ellipse)
            E_m = 1.0, # Young's modulus in matrix
            nu_m = 0.35, # Poisson's ratio in matrix
            E_i = 11.0, # Young's modulus of inclusion
            nu_i = nu_i_values[i], # Poisson's ratio in inclusion
            mesh_size = mesh_values[j],
            mesh_order = 1 )
    
        L2_sol = np.sqrt( dolfinx.fem.assemble_scalar(dolfinx.fem.form( ufl.dot(u_solution,u_solution)* ufl.dx)) )
        L2_ref = np.sqrt( dolfinx.fem.assemble_scalar(dolfinx.fem.form( ufl.dot(u_ref_func,u_ref_func)* ufl.dx)) )
        print(L2_sol)
        print(L2_ref)
        error_matrix[j][i]=error
    
print(error_matrix)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Ellipse)
Info    : [ 50%] Meshing curve 2 (Ellipse)
Info    : Done meshing 1D (Wall 0.000880289s, CPU 0.00155s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 50%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0233097s, CPU 0.024185s)
Info    : 232 nodes 471 elements
58.81705561458823
58.83891107138668
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Ellipse)
Info    : [ 50%] Meshing curve 2 (Ellipse)
Info    : Done meshing 1D (Wall 0.000719876s, CPU 0.002016s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 50%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0544078s, CPU 0.091518s)
Info    : 801 nodes 1615 elements
58.983314276364204
59.000074721848975
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Ellipse)
Info    : [ 50%] Meshing curve 2 (Ellipse

Info    : [ 50%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.546302s, CPU 0.548483s)
Info    : 11393 nodes 22837 elements
59.06066699066668
59.062776566161794
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Ellipse)
Info    : [ 50%] Meshing curve 2 (Ellipse)
Info    : Done meshing 1D (Wall 0.000848645s, CPU 0.000988s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 50%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 2.37754s, CPU 2.37838s)
Info    : 44849 nodes 89799 elements
59.06494366562055
59.065446311042116
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Ellipse)
Info    : [ 50%] Meshing curve 2 (Ellipse)
Info    : Done meshing 1D (Wall 0.000343555s, CPU 0.000454s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 50%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.00749874s